> Note: Before running the Metrics & Coverage cell, generate fresh coverage artifacts.
>
> - Preferred: `nox -s test`
> - Or: `pytest --cov=src --cov-report=xml`
>
> The metrics step expects `coverage.xml` at the repository root. If missing, run one of the above first.

# X Agent Unified Repository Inspection & Dry-Run Notebook

This notebook performs a structured, offline inspection and smoke validation of the Play-stuff (Unified X Agent) repository.

Outline Covered:
1. Setup (env + deps)
2. GitHub tree fetch (optional offline if internet disabled) / local file enumeration
3. Parse Python modules (AST) & build dependency graph
4. ASCII + formatting checks
5. Config schema validation
6. Telemetry init (noop vs enabled)
7. SQLite storage initialization & inspection
8. BudgetManager simulation
9. RateLimiter simulation
10. XClient dry-run stubbed calls
11. Thompson Sampling learning simulation
12. Scheduler end-to-end dry-run
13. Subset pytest execution
14. Persist summary artifacts

All network/API calls are stubbed or dry-run safe; no secrets required.

In [ ]:
# 1. Setup: Environment and Dependencies
import os, sys, json, subprocess, textwrap, pathlib, shutil
from pathlib import Path

PROJECT_ROOT = Path('..').resolve().parent if Path('.').name == 'notebooks' else Path('.').resolve()
SRC_DIR = PROJECT_ROOT / 'src'
TESTS_DIR = PROJECT_ROOT / 'tests'

# Ensure src is on PYTHONPATH
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

# Simulated dependency install (commented to avoid actual pip in notebook execution)
print('[INFO] Environment prepared (simulated).')
print('[INFO] PYTHONPATH entries:', sys.path[:3])

# Placeholder .env content (not written for safety)
sample_env = textwrap.dedent('''\
X_AUTH_MODE=tweepy
X_API_KEY=FAKE
X_API_KEY_SECRET=FAKE
X_ACCESS_TOKEN=FAKE
X_ACCESS_TOKEN_SECRET=FAKE
ENABLE_TELEMETRY=false
''')
print(sample_env)

In [ ]:
# 2. Fetch Repository Tree (local enumeration fallback)
import json, os
repo_tree = []
for root, dirs, files in os.walk(PROJECT_ROOT):
    rel_root = os.path.relpath(root, PROJECT_ROOT)
    if rel_root.startswith('.git') or rel_root.startswith('__pycache__'):
        continue
    for f in files:
        repo_tree.append(os.path.join(rel_root, f))
print('[INFO] Total files discovered:', len(repo_tree))
print('[INFO] Sample:', repo_tree[:15])

with open('repo_tree.json','w') as fh:
    json.dump(repo_tree, fh, indent=2)
print('[ARTIFACT] repo_tree.json written')

In [ ]:
# 3. Parse Python Modules and Map Architecture
import ast, networkx as nx
from collections import defaultdict

def analyze_module(path: Path):
    try:
        src = path.read_text(encoding='utf-8')
    except Exception as e:
        return None
    try:
        tree = ast.parse(src)
    except SyntaxError:
        return None
    mod = {
        'path': str(path),
        'imports': [],
        'defs': [],
        'classes': [],
    }
    for node in ast.walk(tree):
        if isinstance(node, ast.Import):
            for n in node.names:
                mod['imports'].append(n.name.split('.')[0])
        elif isinstance(node, ast.ImportFrom):
            if node.module:
                mod['imports'].append(node.module.split('.')[0])
        elif isinstance(node, ast.FunctionDef):
            mod['defs'].append(node.name)
        elif isinstance(node, ast.ClassDef):
            mod['classes'].append(node.name)
    return mod

modules = []
for py in SRC_DIR.rglob('*.py'):
    modules.append(analyze_module(py))
modules = [m for m in modules if m]
print('[INFO] Parsed modules:', len(modules))

# Simple dependency graph
G = nx.DiGraph()
for m in modules:
    mod_name = Path(m['path']).stem
    G.add_node(mod_name)
    for imp in m['imports']:
        if imp in {'src','tests'}:
            continue
        G.add_edge(mod_name, imp)
print('[INFO] Graph nodes:', len(G.nodes))
print('[INFO] Graph edges:', len(G.edges))

# Persist graph data
graph_data = {'nodes': list(G.nodes), 'edges': list(G.edges)}
with open('module_graph.json','w') as fh:
    json.dump(graph_data, fh, indent=2)
print('[ARTIFACT] module_graph.json written')

In [ ]:
# 4. Run ASCII and Formatting Checks (simulated)
ascii_violations = []
def is_ascii(s: str):
    try:
        s.encode('ascii')
        return True
    except UnicodeEncodeError:
        return False

for py in SRC_DIR.rglob('*.py'):
    content = py.read_text(encoding='utf-8', errors='ignore')
    if not is_ascii(content):
        ascii_violations.append(str(py))

print('[INFO] ASCII violations:', ascii_violations if ascii_violations else 'None')
# Ruff simulation (no real invocation here)
ruff_result = {'summary':'Simulated check passed','issues':[]}
print('[INFO] Ruff formatting/lint (simulated):', ruff_result)

with open('lint_ascii_summary.json','w') as fh:
    json.dump({'ascii_violations': ascii_violations,'ruff': ruff_result}, fh, indent=2)
print('[ARTIFACT] lint_ascii_summary.json written')

## 6. Config Schema Validation

Validate the main config file (`config.yaml`) against the strict schema in `src/config_schema.py`.
- Check for required fields, types, and value constraints.
- Summarize any errors or warnings.
- Output: `config_validation_summary.json` (for downstream cells).

---


In [ ]:
# 6. Config Schema Validation
import json
from pathlib import Path
from pydantic import ValidationError
import sys
sys.path.append(str(Path('src').resolve()))
from config_schema import UnifiedConfig

# Load config file
config_path = Path('config.yaml')
if not config_path.exists():
    config_path = Path('config.example.yaml')

try:
    with open(config_path, 'r', encoding='utf-8') as f:
        import yaml
        config_data = yaml.safe_load(f)
    config = UnifiedConfig(**config_data)
    result = {'valid': True, 'errors': None}
except (ValidationError, Exception) as e:
    result = {'valid': False, 'errors': str(e)}

with open('config_validation_summary.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, indent=2)

print('Config validation complete. See config_validation_summary.json.')


## 7. Telemetry & Tracing Validation

Check if OpenTelemetry tracing is enabled and functioning:
- Inspect environment variables and config for telemetry settings.
- Attempt to start a span using `telemetry.start_span`.
- Summarize results and any errors.
- Output: `telemetry_validation_summary.json`.

---


In [ ]:
# 7. Telemetry & Tracing Validation
import json
import os
from pathlib import Path
sys.path.append(str(Path('src').resolve()))
import telemetry

telemetry_enabled = os.environ.get('ENABLE_TELEMETRY', 'false').lower() == 'true'
span_result = None
error = None
try:
    with telemetry.start_span('notebook-validation-span') as span:
        span_result = str(span)
except Exception as e:
    error = str(e)

result = {
    'telemetry_enabled': telemetry_enabled,
    'span_result': span_result,
    'error': error
}
with open('telemetry_validation_summary.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, indent=2)
print('Telemetry validation complete. See telemetry_validation_summary.json.')


## 8. Storage Validation

Validate SQLite storage and key tables:
- Check existence and schema of `data/agent_unified.db`.
- Inspect action log, metrics, dedup tables, and bandit tables.
- Summarize row counts and schema health.
- Output: `storage_validation_summary.json`.

---


In [ ]:
# 8. Storage Validation
import sqlite3
import json
from pathlib import Path

db_path = Path('data/agent_unified.db')
result = {'db_exists': db_path.exists(), 'tables': {}, 'error': None}
if db_path.exists():
    try:
        conn = sqlite3.connect(str(db_path))
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = [row[0] for row in cursor.fetchall()]
        for table in tables:
            cursor.execute(f"PRAGMA table_info({table});")
            schema = cursor.fetchall()
            cursor.execute(f"SELECT COUNT(*) FROM {table};")
            row_count = cursor.fetchone()[0]
            result['tables'][table] = {
                'schema': schema,
                'row_count': row_count
            }
        conn.close()
    except Exception as e:
        result['error'] = str(e)
else:
    result['error'] = 'Database file not found.'

with open('storage_validation_summary.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, indent=2)
print('Storage validation complete. See storage_validation_summary.json.')


## 9. Budget Validation

Validate budget logic and safety buffers:
- Check plan caps and buffer enforcement in `src/budget.py`.
- Simulate a usage scenario and verify correct budget status.
- Output: `budget_validation_summary.json`.

---


In [ ]:
# 9. Budget Validation
import json
from pathlib import Path
sys.path.append(str(Path('src').resolve()))
import budget

# Simulate usage scenario
try:
    # Example: free plan, 100 actions, 10MB storage
    plan = 'free'
    actions = 100
    storage_mb = 10
    status = budget.check_budget(plan=plan, actions=actions, storage_mb=storage_mb)
    result = {'valid': True, 'status': status}
except Exception as e:
    result = {'valid': False, 'error': str(e)}

with open('budget_validation_summary.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, indent=2)
print('Budget validation complete. See budget_validation_summary.json.')


## 10. Rate Limiter Validation

Validate per-endpoint rate limiting and backoff logic:
- Simulate requests to key endpoints using `src/rate_limiter.py`.
- Check backoff, jitter, and reset logic.
- Output: `rate_limiter_validation_summary.json`.

---


In [ ]:
# 10. Rate Limiter Validation
import json
from pathlib import Path
sys.path.append(str(Path('src').resolve()))
import rate_limiter

result = {}
try:
    # Simulate requests to a test endpoint
    endpoint = '/2/tweets'
    rl = rate_limiter.RateLimiter()
    # Simulate 5 requests
    for i in range(5):
        allowed, wait = rl.check(endpoint)
        result[f'request_{i+1}'] = {'allowed': allowed, 'wait': wait}
    result['valid'] = True
except Exception as e:
    result = {'valid': False, 'error': str(e)}

with open('rate_limiter_validation_summary.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, indent=2)
print('Rate limiter validation complete. See rate_limiter_validation_summary.json.')


## 11. XClient Dry-Run Validation

Run a dry-run using `src/x_client.py` and `src/main.py`:
- Simulate posting and liking actions in both Tweepy and OAuth2 modes.
- Ensure no real network calls are made and dry-run logic is respected.
- Output: `xclient_dryrun_summary.json`.

---


In [ ]:
# 11. XClient Dry-Run Validation
import json
from pathlib import Path
sys.path.append(str(Path('src').resolve()))
from x_client import XClient
from auth import UnifiedAuth

results = {}
for mode in ['tweepy', 'oauth2']:
    try:
        auth = UnifiedAuth(mode=mode)
        client = XClient(auth=auth, dry_run=True)
        post_result = client.post_tweet('Dry-run test tweet', media=None)
        like_result = client.like_tweet('1234567890')
        results[mode] = {
            'post_result': str(post_result),
            'like_result': str(like_result)
        }
    except Exception as e:
        results[mode] = {'error': str(e)}

with open('xclient_dryrun_summary.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=2)
print('XClient dry-run validation complete. See xclient_dryrun_summary.json.')


## 12. Learning Loop Validation

Validate Thompson Sampling and bandit updates:
- Simulate arm selection and metric update using `src/learn.py`.
- Output: `learning_validation_summary.json`.

---


In [ ]:
# 12. Learning Loop Validation
import json
from pathlib import Path
sys.path.append(str(Path('src').resolve()))
import learn

result = {}
try:
    # Simulate arm selection and metric update
    arms = [('topic1', 'morning', 'image'), ('topic2', 'evening', 'none')]
    selected = learn.select_arm(arms)
    learn.update_metrics(selected, reward=1.0)
    result = {'selected_arm': selected, 'update': 'success'}
except Exception as e:
    result = {'error': str(e)}

with open('learning_validation_summary.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, indent=2)
print('Learning loop validation complete. See learning_validation_summary.json.')


## 13. Scheduler Validation

Validate time-window posting and orchestration:
- Simulate scheduling actions using `src/scheduler.py`.
- Output: `scheduler_validation_summary.json`.

---


In [ ]:
# 13. Scheduler Validation
import json
from pathlib import Path
sys.path.append(str(Path('src').resolve()))
import scheduler

result = {}
try:
    # Simulate scheduling a post for morning window
    action = {'type': 'post', 'window': 'morning', 'content': 'Test scheduled tweet'}
    scheduled = scheduler.schedule_action(action)
    result = {'scheduled_action': scheduled}
except Exception as e:
    result = {'error': str(e)}

with open('scheduler_validation_summary.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, indent=2)
print('Scheduler validation complete. See scheduler_validation_summary.json.')


## 14. Pytest Subset Validation

Run a subset of unit tests for key modules:
- Use `pytest` to run tests for `auth`, `budget`, `rate_limiter`, `storage`, and `x_client`.
- Summarize results and failures.
- Output: `pytest_subset_summary.json`.

---


In [ ]:
# 14. Pytest Subset Validation
import json
import subprocess
from pathlib import Path

test_files = [
    'tests/test_auth.py',
    'tests/test_budget.py',
    'tests/test_rate_limiter.py',
    'tests/test_storage.py',
    'tests/test_x_client.py'
]
results = {}
for test_file in test_files:
    try:
        proc = subprocess.run(['pytest', '-v', test_file], capture_output=True, text=True)
        results[test_file] = {
            'returncode': proc.returncode,
            'stdout': proc.stdout,
            'stderr': proc.stderr
        }
    except Exception as e:
        results[test_file] = {'error': str(e)}

with open('pytest_subset_summary.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=2)
print('Pytest subset validation complete. See pytest_subset_summary.json.')


## 15. Artifact Export & Summary

Export all validation summaries and key artifacts:
- List all `*_summary.json` files generated in this notebook.
- Provide a summary table of validation results.
- Output: `notebook_artifact_index.json`.

---


In [ ]:
# 15. Artifact Export & Summary
import json
from pathlib import Path

artifact_files = list(Path('.').glob('*_summary.json'))
index = {}
for file in artifact_files:
    try:
        with open(file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        index[file.name] = data
    except Exception as e:
        index[file.name] = {'error': str(e)}

with open('notebook_artifact_index.json', 'w', encoding='utf-8') as f:
    json.dump(index, f, indent=2)
print('Artifact export complete. See notebook_artifact_index.json.')


# Detailed Breakdown & Action Plan

## 1. Improvement Opportunities

**Modularization**
- Identify and refactor tightly coupled logic in `x_client.py` and `actions.py`.
- Separate API, business logic, and orchestration layers.
- Add unit tests for new modules.

**Telemetry Expansion**
- Ensure all critical actions and errors are traced.
- Add granular spans for key operations (auth, post, like, rate limit, storage).
- Integrate OpenTelemetry context into logs.

**Rate Limiter Integration**
- Deepen integration of rate limiter with all client actions.
- Log rate limit events and backoff decisions.
- Add tests for edge cases and failures.

**Security Hardening**
- Review token handling and storage.
- Audit error logging for sensitive data leaks.
- Harden external API boundaries and input validation.

**Learning Loop Refinement**
- Tune Thompson Sampling parameters for better exploration/exploitation.
- Add more robust bandit metrics and logging.
- Test learning loop under varied scenarios.

---

## 2. Prioritization

**Quick Wins**
- Telemetry expansion
- Rate limiter integration
- Error logging improvements

**Medium Effort**
- Modularization
- Security hardening

**Strategic**
- Learning loop enhancements
- Advanced config validation
- Multi-region storage

---

## 3. Actionable Checklists

### Modularization
- [ ] List all tightly coupled functions/classes in `x_client.py` and `actions.py`
- [ ] Refactor into separate modules
- [ ] Add/expand unit tests
- [ ] Document new module boundaries

### Telemetry Expansion
- [ ] Audit current tracing coverage
- [ ] Add spans for missing operations
- [ ] Integrate trace context into logs
- [ ] Validate with OpenTelemetry collector

### Rate Limiter Integration
- [ ] Map all client actions to rate limiter logic
- [ ] Add logging for rate limit events
- [ ] Test backoff/jitter scenarios
- [ ] Document integration points

### Security Hardening
- [ ] Review token handling/storage
- [ ] Audit error logs for sensitive data
- [ ] Harden API input validation
- [ ] Add security-focused tests

### Learning Loop Refinement
- [ ] Review Thompson Sampling parameters
- [ ] Add/expand bandit metrics
- [ ] Test learning loop under edge cases
- [ ] Document learning logic

---

## 4. Metrics & Coverage

Add a code cell to summarize test coverage, test counts, and risk hotspots by module using pytest and coverage data.


In [ ]:
# 16. Metrics & Coverage Summary
import json
import subprocess
from pathlib import Path

# Run pytest with coverage
proc = subprocess.run(['pytest', '--cov=src', '--cov-report=json', '--maxfail=1', '--disable-warnings'], capture_output=True, text=True)
coverage_file = Path('coverage.json')
coverage_data = {}
if coverage_file.exists():
    with open(coverage_file, 'r', encoding='utf-8') as f:
        coverage_data = json.load(f)
else:
    coverage_data = {'error': 'coverage.json not found'}

# Summarize test counts and risk hotspots
summary = {
    'pytest_stdout': proc.stdout,
    'pytest_stderr': proc.stderr,
    'coverage': coverage_data
}
with open('metrics_coverage_summary.json', 'w', encoding='utf-8') as f:
    json.dump(summary, f, indent=2)
print('Metrics & coverage summary complete. See metrics_coverage_summary.json.')
